In [1]:
import os
import sys

print(os.getcwd())
cwd0 = './config/'
sys.path.append(cwd0)

import visualID_Eng as vID
from visualID_Eng import color
vID.init(cwd0)
import tools4PytChem as t4p

/home/romuald/ENSEIGNEMENT/0-JupyterNotebooks/PytChem


**Start at:** Friday 11 November 2022, 21:11:27  
**Hostname:** ip-sw-install-nico.insa-toulouse.fr (Linux)

<p style="text-align: center"><img width="800px" src="./config/svg/logoPytChem.svg" style="margin-left:auto; margin-right:auto"/></p>

# Particle in a box (infinite square well)

## 1D quantum particle in a box

In [13]:
import sympy as sym
from tools4PytChem import PrintLatexStyleSymPyEquation as ple
